### IMPORTS

In [1]:
import base64
import hashlib
import subprocess
import time
import binascii

In [2]:
import ipynb.fs.full.utils as utils
import ipynb.fs.full.blockchain as blockchain

### FIX FOR WINDOWS ENVIRONMENT

In [3]:
#import pkg_resources.py2_warn

### MIDDLEWARE STARTS

In [4]:
print('BOOTING UP...')

BOOTING UP...


### LOAD DATA FROM EXTERNAL FILES

In [5]:
settings = utils.load_json('config/settings.json')

In [6]:
latest = utils.load_json('config/latest.json')

In [7]:
print('READ EXTERNAL FILES!')

READ EXTERNAL FILES!


### EXTRACT PUBLIC/PRIVATE KEY FROM FILE

In [8]:
def extract(path):
    
    # OPEN THE FILE
    with open(path) as file:
        encrypted_key = file.read()
        
        # DECRYPT & CONVERT FROM HEXBYTES
        private_key = web3.eth.account.decrypt(encrypted_key, 'pass')
        hexed = web3.toHex(private_key)
        
        # PARSE FILE AS JSON & EXTRACT THE ADDRESS
        obj = json.loads(encrypted_key)
        address = '0x' + obj['address']
        
        # GENERATE THE CHECKSUM VERSION
        checksum = web3.toChecksumAddress(address)
        
        return {
            'public': checksum,
            'private': hexed
        }

### CONNECT TO BLOCKCHAIN VIA WEBSOCKET

In [9]:
web3 = blockchain.connect(settings)

CONNECTED WITH GATEWAY!


### SERIALIZE MANAGER CONTRACTS

In [10]:
user_manager = blockchain.contract(latest['usermanager'], web3, settings)

In [11]:
oracle_manager = blockchain.contract(latest['oraclemanager'], web3, settings)

In [12]:
task_manager = blockchain.contract(latest['taskmanager'], web3, settings)

In [13]:
token_manager = blockchain.contract(latest['tokenmanager'], web3, settings)

In [14]:
print('MANAGER CONTRACTS SERIALIZED!')

MANAGER CONTRACTS SERIALIZED!


### INIT MANAGER CONTRACTS

In [15]:
def init_managers():
    
    # TOKEN PARAMS
    token_symbol = 'ArcaCoin'
    token_price = 5000
    token_capacity = 10000
    
    # TASK PARAMS
    task_fee = 2
    
    # TOKEN MANAGER
    token_manager.write({
        'func': 'init',
        'params': [
            token_symbol,
            token_price,
            token_capacity,
            task_manager.address
        ]
    })
    
    print('TOKEN MANAGER INITIALIZED')
    
    # TASK MANAGER
    task_manager.write({
        'func': 'init',
        'params': [
            task_fee,
            user_manager.address,
            oracle_manager.address,
            token_manager.address
        ]
    })
    
    print('TASK MANAGER INITIALIZED')
    
    # USER MANAGER
    user_manager.write({
        'func': 'init',
        'params': [task_manager.address]
    })
    
    print('USER MANAGER INITIALIZED')
    
    # DEVICE MANAGER
    oracle_manager.write({
        'func': 'init',
        'params': [
            user_manager.address,
            task_manager.address
        ]
    })
    
    print('ORACLE MANAGER INITIALIZED')
    
    return True

### INITIALIZE MANAGERS IF NECESSARY

In [18]:
def check():
    
    # CHECK STATUS
    initialized = task_manager.read('initialized')
    
    # INIT HAS BEEN PERFORMED
    if (initialized):
        print('MANAGER CONTRACTS HAVE ALREADY BEEN INITIALIZED!')

    # INIT NOW
    else:
        result = init_managers()

        if (result):
            print('\n INITIALIZATION COMPLETED!')

In [19]:
check()

MANAGER CONTRACTS HAVE ALREADY BEEN INITIALIZED!


### CHECK AGAIN TO CONFIRM

task_manager.write({
    'func': 'add',
    'params': [
        'foobar',
        'ea89158adbe59aa9c5e8147c50f2e5cccc3a0d565202d58abc57d5c8946b78e7',
        1,
        'dsfsdfdsfs',
        5
    ]
})